In [2]:
import pandas as pd
import ast
import os


# Reading dataset

In [3]:
path = "../instagram_data.csv"
data =pd.read_csv(path)
data.head()

,owner_id,owner_username,shortcode,is_video,caption,comments,likes,created_at,location,imageUrl,multiple_images,username,followers,following
0,36063641,christendominique,C3_GS1ASeWI,False,I’m a brunch & Iced Coffee girlie☕️🍳 \n\nTop @...,268,16382,1.709327e+09,NaN,https://instagram.flba2-1.fna.fbcdn.net/v/t39....,True,christendominique,2144626.0,1021.0
1,36063641,christendominique,C38ivgNS3IX,True,😮‍💨Brow tips I really wish I would have know w...,138,9267,1.709241e+09,NaN,https://instagram.flba2-1.fna.fbcdn.net/v/t51....,False,christendominique,2144626.0,1021.0
2,36063641,christendominique,C35-Dd9SO1b,True,OMG I can’t believe it’s already been 1 yr sin...,1089,10100,1.709155e+09,NaN,https://instagram.flba2-1.fna.fbcdn.net/v/t51....,False,christendominique,2144626.0,1021.0
3,36063641,christendominique,C33TadDMisq,True,90’s Glam was Pam! \n\nMakeup \n@smashboxcosme...,271,6943,1.709065e+09,NaN,https://instagram.flba2-1.fna.fbcdn.net/v/t51....,False,christendominique,2144626.0,1021.0
4,36063641,christendominique,C3s-Cm1yCba,True,Chiseled & Sculptured 🏛️\n\n Contour @westmana...,145,17158,1.708718e+09,NaN,https://instagram.flba2-1.fna.fbcdn.net/v/t51....,False,christendominique,2144626.0,1021.0


In [4]:
data.isna().sum()

owner_id              0
owner_username        2
shortcode             2
is_video              2
caption             161
comments              2
likes                 2
created_at            3
location           7169
imageUrl              3
multiple_images       3
username           2115
followers          2115
following          2115
dtype: int64

In [5]:
data.loc[data.owner_username != data.username]

,owner_id,owner_username,shortcode,is_video,caption,comments,likes,created_at,location,imageUrl,multiple_images,username,followers,following
22,46755025637,mangrovephotographyawards,CuU74i5Krh-,False,👀 Photography is a powerful medium for connect...,1,292,1.688585e+09,NaN,https://instagram.flba2-1.fna.fbcdn.net/v/t51....,True,NaN,NaN,NaN
43,330497544,images_vevey,C3pcxFHoYRY,False,Christopher Anderson - Family Trilogy\n\n📍L’Ap...,11,1977,1.708600e+09,NaN,https://instagram.fgyn2-1.fna.fbcdn.net/v/t39....,True,NaN,NaN,NaN
107,352125728,kameraexpress,Cy-cUTDoBu5,True,@claireonline laat zich graag betoveren door d...,23,800,1.698567e+09,NaN,https://instagram.flba2-1.fna.fbcdn.net/v/t51....,False,NaN,NaN,NaN
182,5803476947,phu_srpa,C2aC7LuJJNa,False,Today’s an auspicious day . We KCC family feel...,5,766,1.705936e+09,"{'id': '880072107', 'has_public_page': True, '...",https://instagram.fmar3-1.fna.fbcdn.net/v/t39....,True,NaN,NaN,NaN
184,458305551,petzl_official,C2IGHOFNiSK,True,#PetzlTips - Organize your ice screws with CAR...,25,3448,1.705334e+09,NaN,https://instagram.fmar3-1.fna.fbcdn.net/v/t51....,False,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11660,=======,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11661,"📸 @itslitt_photography""",37,2683,1707433865,"{'id': '8627549', 'has_public_page': True, 'na...",https://instagram.flba2-1.fna.fbcdn.net/v/t39....,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11662,239264125,nightaccess,C3DuDcpuLjX,False,If you missed @madds and friends at @parqsd we...,13,587,1.707334e+09,"{'id': '341620970', 'has_public_page': True, '...",https://instagram.flba2-1.fna.fbcdn.net/v/t51....,True,NaN,NaN,NaN
11664,19681605,samblacky,C25XVf2v0yp,True,We’re Female DJs 👊🏼 \nHere is just a few of ou...,866,4899,1.706987e+09,NaN,https://instagram.flba2-1.fna.fbcdn.net/v/t51....,False,NaN,NaN,NaN


In [6]:
data.isna().sum()

owner_id              0
owner_username        2
shortcode             2
is_video              2
caption             161
comments              2
likes                 2
created_at            3
location           7169
imageUrl              3
multiple_images       3
username           2115
followers          2115
following          2115
dtype: int64

# Locational Data
data rows with filled location values

In [7]:
locational_data = data.loc[data.location.isna() != True]
locational_data.head(1)

,owner_id,owner_username,shortcode,is_video,caption,comments,likes,created_at,location,imageUrl,multiple_images,username,followers,following
28,40506412,christinehmcconnell,CzJtDjegmdI,False,Had an amazing time in #salem for Halloween th...,466,79000,1.698945e+09,"{'id': '34537076', 'has_public_page': True, 'n...",https://instagram.ffor43-1.fna.fbcdn.net/v/t39...,True,christinehmcconnell,597189.0,1576.0


*`coverting all values to dictionary`*

In [8]:
def dict_maker(row):
    return ast.literal_eval(row["location"])

locational_data["location"] = locational_data.apply(dict_maker, axis=1)

C:\Users\saeed\AppData\Local\Temp\ipykernel_1940\1118611153.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  locational_data["location"] = locational_data.apply(dict_maker, axis=1)


*`getting all city names in dictionary, and then replace the final list of city names with current column of location`*

In [9]:
location = []
for key in locational_data.location:
    location.append(key["slug"])

# replace old colum with new list
locational_data.location = location

C:\Users\saeed\AppData\Local\Temp\ipykernel_1940\762568345.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  locational_data.location = location


In [10]:
locational_data = locational_data[locational_data['username'].notna()]
locational_data.head(1)

,owner_id,owner_username,shortcode,is_video,caption,comments,likes,created_at,location,imageUrl,multiple_images,username,followers,following
28,40506412,christinehmcconnell,CzJtDjegmdI,False,Had an amazing time in #salem for Halloween th...,466,79000,1.698945e+09,salem-massachusetts,https://instagram.ffor43-1.fna.fbcdn.net/v/t39...,True,christinehmcconnell,597189.0,1576.0


convert this to dataset "Locational_dataset" 

----

# Non-Locational Data

In [11]:
nonlocational_data = data.loc[data.username.isna() == False]
nonlocational_data.drop(columns="location", inplace=True)

C:\Users\saeed\AppData\Local\Temp\ipykernel_1940\2266298829.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nonlocational_data.drop(columns="location", inplace=True)


In [12]:
nonlocational_data.head(1)

,owner_id,owner_username,shortcode,is_video,caption,comments,likes,created_at,imageUrl,multiple_images,username,followers,following
0,36063641,christendominique,C3_GS1ASeWI,False,I’m a brunch & Iced Coffee girlie☕️🍳 \n\nTop @...,268,16382,1.709327e+09,https://instagram.flba2-1.fna.fbcdn.net/v/t39....,True,christendominique,2144626.0,1021.0


In [13]:
nonlocational_data.isna().sum()

owner_id             0
owner_username       0
shortcode            0
is_video             0
caption            136
comments             0
likes                0
created_at           0
imageUrl             0
multiple_images      0
username             0
followers            0
following            0
dtype: int64

---

# Exporting Data 
Exporting Data on csv file in `prepare data` folder

In [14]:
folder_path = '../Prepared Data'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Save each DataFrame to a CSV file in the new folder
nonlocational_data.to_csv(os.path.join(folder_path, 'nonlocational_data.csv'), index=False)
locational_data.to_csv(os.path.join(folder_path, 'locational_data.csv'), index=False)
